<a href="https://colab.research.google.com/github/mofurinosuke/Post-Quantum-Cryptography-LWE/blob/main/%E5%AF%BE%E9%87%8F%E5%AD%90%E6%9A%97%E5%8F%B7%E3%81%AE%E5%AE%9F%E8%B7%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 格子暗号（LWE）通信フロー図

| 手順 | 受信側 (Server) | 送信側 (MyComputer) |
| :--- | :--- | :--- |
| **1. 鍵生成** | **秘密鍵 $s=\begin{pmatrix} 2 \\ 2 \end{pmatrix}$, 誤差 $e=\begin{pmatrix} 1 \\ -1 \end{pmatrix}$**<br>公開鍵 $b = As + e = \begin{pmatrix} 31 \\ 9 \end{pmatrix}$ | |
| **2. 共有** | 公開鍵 $(A, b)$ を公開 $\rightarrow$ | 公開鍵を取得 |
| **3. 暗号化** | | [cite_start]**メッセージ "m" [cite: 1]**<br>$M = 20 \times 10 = 200$<br>乱数 $r=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$, 誤差 $e_1, e_2$ |
| **4. 計算** | | $u = A^T r + e_1 = \begin{pmatrix} 11 \\ 4 \end{pmatrix}$<br>$v = b^T r + M + e_2 = 232$ |
| **5. 送信** | $\leftarrow$ 暗号文 $(u, v)$ を受信 | 暗号文 $(u, v)$ を送信 |
| **6. 復号** | $v - s^T u = 232 - 30 = 202$ | |
| **7. 完了** | [cite_start]$202 \div 10 \approx 20$ **("m") [cite: 1]** | **成功** |

---
#実践用プログラム
## LWEモデルのパラメータ

### システム共通設定
* **法 $q$**: `10000` （演算空間の定義）
* **スケーリング $\Delta$**: `100` （メッセージ増幅幅）
* **公開行列 $A$**: **$4 \times 4$ 行列**
    * 生成：各実行時に $[0, q)$ の範囲でランダム生成

### 生成規則（毎回動的に生成）
| 要素 | 形状 | 値の範囲 / 生成規則 | 秘匿性 |
| :--- | :--- | :--- | :--- |
| **秘密鍵 $s$** | $4 \times 1$ | 整数 $[0, 4]$ | **秘密** |
| **誤差 $e, e_1, e_2$** | ベクトル/スカラ | 整数 $[-2, 2]$ | **秘密** |
| **一時乱数 $r$** | $4 \times 1$ | バイナリ $\{0, 1\}$ | **秘密 (使い捨て)** |

### 出力データ (通信対象)
* **暗号文 $u$**: $4 \times 1$ ベクトル ($u = A^T r + e_1 \pmod q$)
* **暗号文 $v$**: スカラー ($v = b^T r + M + e_2 \pmod q$)

---


In [1]:
import numpy as np

# --- 【設定：世界のルール（System Parameters）】 ---
n = 4          # 次元の数（2だと簡単すぎるので4次元にします）
q = 10000       # 法（Modulus）：すべての計算はこの世界の中で行われます
delta = 100    # スケーリング係数：ノイズに埋もれないよう、メッセージを100倍します

# --- 【受信側：鍵の生成（Key Generation）】 ---
print("--- 1. Receiver: Key Generation ---")

# 1. 公開行列 A (n x n): 完全にランダムな行列
A = np.random.randint(0, q, (n, n))

# 2. 秘密鍵 s (n x 1): 小さな値のベクトル
s = np.random.randint(0, 5, (n, 1))

# 3. 誤差 e (n x 1): 小さなノイズ
e = np.random.randint(-2, 3, (n, 1))

# 4. 公開鍵 b の計算 (LWEの方程式: b = As + e)
b = (np.dot(A, s) + e) % q

print(f"秘密鍵 s (Secret):\n{s.T} (転置表示)")
print(f"公開鍵 b (Public):\n{b.T} (転置表示)")
print(f"公開行列 A (の一部):\n{A[:2]} ...")

--- 1. Receiver: Key Generation ---
秘密鍵 s (Secret):
[[0 0 2 4]] (転置表示)
公開鍵 b (Public):
[[3246  768 3239 7190]] (転置表示)
公開行列 A (の一部):
[[1968 1281 6956 2334]
 [4102 1882 7557 1414]] ...


In [2]:
# --- 【送信側：暗号化（Encryption）】 ---
print("\n--- 2. Sender: Encryption ---")

# 1. 送りたいメッセージ（例：数字の 7）
m_original = 7
M = m_original * delta  # スケーリング (7 * 100 = 700)
print(f"平文 m: {m_original} -> スケーリング済み M: {M}")

# 2. 一時的な乱数 r (n x 1): 今回限りの使い捨て鍵
r = np.random.randint(0, 2, (n, 1))

# 3. 送信側の誤差（ここが重要）
# uを守るための誤差 e1 と、vに乗る誤差 e2
e1 = np.random.randint(-2, 3, (n, 1))
e2 = np.random.randint(-2, 3) # スカラー

# 4. 暗号文 u の計算 (u = A^T r + e1)
# 転置行列 A.T を使うのがポイント
u = (np.dot(A.T, r) + e1) % q

# 5. 暗号文 v の計算 (v = b^T r + e2 + M)
v_temp = np.dot(b.T, r) + e2 + M
v = v_temp % q

print(f"使用した乱数 r:\n{r.T}")
print(f"生成された暗号文 u:\n{u.T}")
print(f"生成された暗号文 v:\n{v}")
print("--> 暗号文 (u, v) を送信します")


--- 2. Sender: Encryption ---
平文 m: 7 -> スケーリング済み M: 700
使用した乱数 r:
[[1 0 0 1]]
生成された暗号文 u:
[[6250 6461 3890 3161]]
生成された暗号文 v:
[[1137]]
--> 暗号文 (u, v) を送信します


In [3]:
# --- 【受信側：復号（Decryption）】 ---
print("\n--- 3. Receiver: Decryption ---")

# 1. マスクの除去 (Result = v - s^T u)
# 数式通りの行列演算
mask = np.dot(s.T, u)
noisy_m = (v - mask) % q

# Pythonの剰余演算は負の数を正に戻してしまうため、
# q/2 (500) を超える場合は「負のノイズ」とみなして調整する処理
noisy_val = noisy_m[0][0]
if noisy_val > q / 2:
    noisy_val -= q

print(f"復号計算の結果 (ノイズ混じり): {noisy_val}")
print(f"正解の数値 (M): {M}")
print(f"誤差の大きさ: {noisy_val - M}")

# 2. 丸め処理（Rounding）
# スケーリング係数(100)で割って四捨五入
decrypted_m = round(noisy_val / delta)

print(f"--- 最終結果 ---")
print(f"復号されたメッセージ: {decrypted_m}")

if decrypted_m == m_original:
    print("SUCCESS: 復号成功！")
else:
    print("FAILED: パラメータ設定を見直してください")


--- 3. Receiver: Decryption ---
復号計算の結果 (ノイズ混じり): 713
正解の数値 (M): 700
誤差の大きさ: 13
--- 最終結果 ---
復号されたメッセージ: 7
SUCCESS: 復号成功！
